# Building training data

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import sqlite3
from pathlib import Path

In [2]:
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
DB_PATH: str = str(DATA_DIR / "ruokasuositusdata.db")
DISHES_PATH: str = str(DATA_DIR / "Ruokasuositusalgoritmi - data.xlsx")

The data about different dishes is stored in an Excel spreadsheet. It has the following columns:
- **Nimi**: Name of the dish
- **Pääraaka-aine**: The main source of protein (chicken, fish etc.)
- **Raskas**: A score of how 'heavy' or 'fatty' the dish tastes on a scale of 1-3 (1=light, 3=heavy)
- **Hinta**: A score of how expensive the dish is on a scale of 1-3 (1=affordable, 3=expensive)
- **Toistuvuus**: A preference score of how often to eat the dish on a scale of 1-4 (1=seldom, 4=often)
- **Kuvaus**: A short description of the dish in English

In [3]:
dishes_df = pd.read_excel(DISHES_PATH)
dishes_df.head()

,Nimi,Pääraaka-aine,Raskas,Hinta,Toistuvuus,Kuvaus
0,Tortillat,Kana,2,2,3,Soft tortillas filled with seasoned chicken an...
1,Muikut,Kala,2,2,2,"Small, lightly fried Finnish vendace fish, typ..."
2,Katkarapupasta,Katkarapu,1,3,1,Pasta tossed with shrimp in a creamy or tomato...
3,Katkarapuwokki,Katkarapu,1,3,1,"Stir-fried shrimp with vegetables and sauce, p..."
4,Kanawokki,Kana,2,2,2,"Stir-fried chicken with vegetables and sauce, ..."


The goal is to predict sets of three dishes.
A training set is constructed by first building all combinations of the available dishes and then sampling a training set.

10 feature columns are constructed to help build a diverse training set.

In [4]:
dummies_df = pd.get_dummies(dishes_df.drop(columns=["Toistuvuus", "Nimi"]),
                   columns=["Pääraaka-aine", "Raskas", "Hinta"])

X = dummies_df.to_numpy()

names = dishes_df["Nimi"].to_numpy()

comb_idx = list(combinations(range(len(names)), 3))

result_matrix = np.array([X[list(idx)].sum(axis=0) for idx in comb_idx])

result_df = pd.DataFrame(result_matrix,
                         columns=dummies_df.columns)

result_df.insert(0, "c", [names[c] for (_, _, c) in comb_idx])
result_df.insert(0, "b", [names[b] for (_, b, _) in comb_idx])
result_df.insert(0, "a", [names[a] for (a, _, _) in comb_idx])

result_df = result_df.drop(columns=["Kuvaus"])

result_df

,a,b,c,Pääraaka-aine_Kala,Pääraaka-aine_Kana,Pääraaka-aine_Kasvis,Pääraaka-aine_Katkarapu,Raskas_1,Raskas_2,Raskas_3,Hinta_1,Hinta_2,Hinta_3
0,Tortillat,Muikut,Katkarapupasta,1,1,0,1,1,2,0,0,2,1
1,Tortillat,Muikut,Katkarapuwokki,1,1,0,1,1,2,0,0,2,1
2,Tortillat,Muikut,Kanawokki,1,2,0,0,0,3,0,0,3,0
3,Tortillat,Muikut,Kalapuikot,2,1,0,0,0,3,0,1,2,0
4,Tortillat,Muikut,Tonnikalapastavuoka,2,1,0,0,0,2,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4955,Paprikapasta,Uuniperunat (skagen),Kanasalaatti,0,1,1,1,2,1,0,1,0,2
4956,Kasvislasagne,Paistetut nuudelit,Uuniperunat (skagen),0,1,1,1,1,2,0,0,2,1
4957,Kasvislasagne,Paistetut nuudelit,Kanasalaatti,0,2,1,0,2,1,0,0,2,1
4958,Kasvislasagne,Uuniperunat (skagen),Kanasalaatti,0,1,1,1,1,2,0,0,1,2


In [5]:
sampled_df = result_df.groupby(result_df.columns[3:].to_list(), group_keys=False).sample(n=1, random_state=0)
sampled_df

,a,b,c,Pääraaka-aine_Kala,Pääraaka-aine_Kana,Pääraaka-aine_Kasvis,Pääraaka-aine_Katkarapu,Raskas_1,Raskas_2,Raskas_3,Hinta_1,Hinta_2,Hinta_3
926,Katkarapupasta,Katkarapuwokki,Uuniperunat (skagen),0,0,0,3,2,1,0,0,0,3
1577,Katkarapuwokki,Gnocchivuoka,Uuniperunat (skagen),0,0,1,2,1,1,1,0,1,2
1562,Katkarapuwokki,Uunifetapasta,Uuniperunat (skagen),0,0,1,2,1,2,0,0,1,2
912,Katkarapupasta,Katkarapuwokki,Gnocchivuoka,0,0,1,2,2,0,1,0,1,2
1151,Katkarapupasta,Vihreä pasta,Uuniperunat (skagen),0,0,1,2,2,1,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,Paistettu kala,Uunilohi,Savulohi,3,0,0,0,1,2,0,0,0,3
879,Muikut,Uunilohi,Savulohi,3,0,0,0,1,2,0,0,1,2
618,Muikut,Kalakeitto,Savulohi,3,0,0,0,1,2,0,0,2,1
2339,Kalapuikot,Uunilohi,Savulohi,3,0,0,0,1,2,0,1,0,2


In [6]:
def create_training_table():
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS training(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        a TEXT,
        b TEXT,
        c TEXT,
        rating INT
        )
        """)
        conn.commit()

In [ ]:
def insert_training_data(df):
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        for i, row in df.iterrows():
            cursor.execute("INSERT INTO training (a, b, c) VALUES (?, ?, ?)", (row["a"], row["b"], row["c"]))
        conn.commit()

In [8]:
def drop_training_table():
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        DROP TABLE training
        """)
        conn.commit()

In [ ]:
def create_dishes_table():
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        CREATE TABLE dishes(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nimi TEXT,
            kuvaus TEXT,
            pääraaka_aine TEXT,
            raskas INT,
            hinta INT,
            toistuvuus INT
            )
        """)
        conn.commit()

def insert_dishes_into_db(df):
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        for i, row in df.iterrows():
            cursor.execute("INSERT INTO dishes (nimi, kuvaus, pääraaka_aine, raskas, hinta, toistuvuus) VALUES (?, ?, ?, ?, ?, ?)", (row["Nimi"], row["Kuvaus"], row["Pääraaka-aine"], row["Raskas"], row["Hinta"], row["Toistuvuus"]))
        conn.commit()

In [ ]:
create_training_table()
create_dishes_table()
insert_training_data(sampled_df)
insert_dishes_into_db(dishes_df)